https://towardsdatascience.com/auto-keras-or-how-you-can-create-a-deep-learning-model-in-4-lines-of-code-b2ba448ccf5e

In [1]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

/Users/robincole/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
%%time
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 118s 2ms/step - loss: 0.2665 - acc: 0.9183 - val_loss: 0.0552 - val_acc: 0.9822
Epoch 2/12
60000/60000 [==============================] - 124s 2ms/step - loss: 0.0895 - acc: 0.9737 - val_loss: 0.0448 - val_acc: 0.9858
Epoch 3/12
60000/60000 [==============================] - 123s 2ms/step - loss: 0.0657 - acc: 0.9801 - val_loss: 0.0343 - val_acc: 0.9871
Epoch 4/12
60000/60000 [==============================] - 123s 2ms/step - loss: 0.0523 - acc: 0.9842 - val_loss: 0.0337 - val_acc: 0.9886
Epoch 5/12
60000/60000 [==============================] - 122s 2ms/step - loss: 0.0457 - acc: 0.9861 - val_loss: 0.0266 - val_acc: 0.9907
Epoch 6/12
60000/60000 [==============================] - 119s 2ms/step - loss: 0.0400 - acc: 0.9880 - val_loss: 0.0271 - val_acc: 0.9907
Epoch 7/12
60000/60000 [==============================] - 123s 2ms/step - loss: 0.0361 - acc: 0.9888 - val_loss: 0.0241 - 

## MNIST with Auto-Keras:

In [4]:
from autokeras.classifier import ImageClassifier

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))

In [5]:
%%time
clf = ImageClassifier(verbose=True, augment=False)
clf.fit(x_train, y_train, time_limit=12 * 60 * 60)
clf.final_fit(x_train, y_train, x_test, y_test, retrain=True)
y = clf.evaluate(x_test, y_test)
print(y * 100)

Initializing search.
Initialization finished.
Training model  0
Saving model.
Model ID: 0
Loss: tensor(1.1985)
Accuracy 99.28
Training model  1
Father ID:  0
[('to_wider_model', 6, 64)]


KeyboardInterrupt: 

Stopped after 1 hour